In this repo we will simply find a quick attempt to use `pagerank` (via `networkx`), i.e. my implementation of `textrank`, to generate some summary of the amazon reviews. 

The amazon reviews are very heterogeneous, some are short, some are long, some are well writen some are certainly not, so we'll see how this works. 

One general comment I want to add is that through the entire repo, whether my experiments work or not, I am going to leave them here, so I share *"unsuccessful code"* as well.

Anyway, having said that let's start by the beginning, breaking the reviews into sentences

In [28]:
import pandas as pd
import gzip
import spacy
import os

from pathlib import Path

In [29]:
n_cpus = os.cpu_count()

DATA_PATH  = Path('../../datasets/amazon_reviews/')
OUT_PATH   = Path('data/')

nlp = spacy.load("en_core_web_sm")
MIN_TOKENS = 5 # min tokens per sentence
MIN_SENTS  = 1 # min sentences per review

For this notebook I am only going to use 1000 reviews

In [30]:
if not os.path.exists(OUT_PATH): os.makedirs(OUT_PATH)
df_org = pd.read_json(DATA_PATH/'reviews_Clothing_Shoes_and_Jewelry_5.json.gz', lines=True)
df_summary = df_org[['reviewText', 'summary']]
df_summary = df_summary[~df_summary.reviewText.isna()].sample(n=1000)

In [31]:
df_summary.head()

,reviewText,summary
69726,"Okay, so they are great jeans for play, if you're playing in the back yard, but they're not as f...","work, not play"
237809,This is a nice set. It can even be worn under other clothes..with a little preparation. Cute out...,Good look
178012,"I bought this for the winter and now that i have them they fit well and they are comfortable, i ...",Good
48519,"I really like the way this wallet holds all my credit cards on one side, and my bills (folded) o...",Does the trick; slightly bulkier than I'd prefer
38814,Bought this for my daughter-in-law after the birth of her 3rd child and it is a beautiful neckla...,Love it!


Note that in this dataset the users actually summarised the reviews themselves. This will be very useful for some supervised, DL-related future techniques I intent to use.

In [32]:
# Keep reviews with more than MIN_TOKENS 'tokens'
keep_review_1 = [len(r.split(' '))>MIN_TOKENS for r in df_summary.reviewText]
df_summary = df_summary.loc[keep_review_1].reset_index(drop=True)

"Sentersizing"!

In [33]:
review_sents = []
for doc in nlp.pipe(df_summary.reviewText.tolist(), n_process=n_cpus, batch_size=1000):
    # to str so it can be pickled
    sents = [str(s) for s in list(doc.sents)]
    review_sents.append(sents)

In [35]:
df_summary['review_sents'] = review_sents
print(df_summary.shape)
df_summary.head()

(993, 3)


,reviewText,summary,review_sents
0,"Okay, so they are great jeans for play, if you're playing in the back yard, but they're not as f...","work, not play","[Okay, so they are great jeans for play, if you're playing in the back yard, but they're not as ..."
1,This is a nice set. It can even be worn under other clothes..with a little preparation. Cute out...,Good look,"[This is a nice set., It can even be worn under other clothes..with a little preparation., Cute ..."
2,"I bought this for the winter and now that i have them they fit well and they are comfortable, i ...",Good,"[I bought this for the winter and now that i have them they fit well, and they are comfortable, ..."
3,"I really like the way this wallet holds all my credit cards on one side, and my bills (folded) o...",Does the trick; slightly bulkier than I'd prefer,"[I really like the way this wallet holds all my credit cards on one side, and my bills (folded) ..."
4,Bought this for my daughter-in-law after the birth of her 3rd child and it is a beautiful neckla...,Love it!,[Bought this for my daughter-in-law after the birth of her 3rd child and it is a beautiful neckl...


In [36]:
keep_review_2 = [len(rs)>MIN_SENTS for rs in review_sents]
df_summary = df_summary.loc[keep_review_2].reset_index(drop=True)
print(df_summary.shape)
df_summary.head()

(940, 3)


,reviewText,summary,review_sents
0,"Okay, so they are great jeans for play, if you're playing in the back yard, but they're not as f...","work, not play","[Okay, so they are great jeans for play, if you're playing in the back yard, but they're not as ..."
1,This is a nice set. It can even be worn under other clothes..with a little preparation. Cute out...,Good look,"[This is a nice set., It can even be worn under other clothes..with a little preparation., Cute ..."
2,"I bought this for the winter and now that i have them they fit well and they are comfortable, i ...",Good,"[I bought this for the winter and now that i have them they fit well, and they are comfortable, ..."
3,"I really like the way this wallet holds all my credit cards on one side, and my bills (folded) o...",Does the trick; slightly bulkier than I'd prefer,"[I really like the way this wallet holds all my credit cards on one side, and my bills (folded) ..."
4,"These are great. Buy multiple pairs ...Pros: Comfortable, lightweight, suit wide head, lenses ou...",Practical,"[These are great., Buy multiple pairs ..., Pros: Comfortable, lightweight, suit wide head, lense..."
